In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import random
import os
from datetime import datetime
# import torch
import sys
sys.path.append('../..')
from modules.many_features import utils, constants
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
SEED = constants.SEED
random.seed(SEED)
np.random.seed(SEED)
os.environ['PYTHONHASHSEED']=str(SEED)
print(SEED)
# torch.manual_seed(SEED)
# torch.use_deterministic_algorithms(True)

0


#### The Data

In [3]:
# sizes = [0.1]
# for size in sizes:
#     train_df = pd.read_csv(f'../../../anemia_ml4hc/data/train_set_basic_{size}_seed_{SEED}.csv')
#     train_df = train_df.fillna(-1)
#     X_train = train_df.iloc[:, 0:-1]
#     y_train = train_df.iloc[:, -1]
#     X_train, y_train = np.array(X_train), np.array(y_train)
#     for steps in [int(15e6)]:
#         dqn_model = utils.stable_double_dqn(X_train, y_train, steps, True, 
#             f'../../../anemia_ml4hc/models/sb/new_different_training_sizes/dqn_pr_train_set_{size}_{SEED}_{steps}')


In [4]:
# missingness_levels = [0.1, 0.2, 0.3]

# for level in missingness_levels:
# # for level in missingness_levels[:1]:
#     train_df = pd.read_csv(f'../../../anemia_ml4hc/data/train_set_missingness_{level}.csv')
#     train_df = train_df.fillna(-1)
# #     print(len(train_df))
#     X_train = train_df.iloc[:, 0:-1]
#     y_train = train_df.iloc[:, -1]
#     X_train, y_train = np.array(X_train), np.array(y_train)
#     print(X_train.shape, y_train.shape)
    
#     for steps in [int(15e6), int(12e6), int(10e6)]:
# #     for steps in [int(2e3)]:
#         dqn_model = utils.stable_dueling_dqn(X_train, y_train, steps, True, 
#         f'../../../anemia_ml4hc/models/sb/new_missingness_noisiness/seed_{SEED}/dueling_dqn_pr_missingness_{level}_{steps}')

In [5]:
level=0.3
train_df = pd.read_csv(f'../../../anemia_ml4hc/data/train_set_missingness_{level}.csv')
train_df = train_df.fillna(-1)
train_df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
0,14.728733,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,1,-1.000000,-1.000000,-1.0,-1.000000,-1.0,-1.000000,44.186200,-1.0,0
1,10.405752,9.634615,5.659537,-1.000000,-1.000000,77.413788,212.671838,4.032519,0,0.887130,96.311597,-1.0,43.218595,-1.0,83.207518,-1.000000,-1.0,4
2,15.132737,-1.000000,1.842252,3.797487,315.102272,80.500314,-1.000000,5.639507,0,-1.000000,-1.000000,-1.0,-1.000000,-1.0,-1.000000,45.398211,-1.0,0
3,11.340169,-1.000000,-1.000000,-1.000000,-1.000000,97.033963,-1.000000,3.506041,1,1.020527,127.281715,-1.0,20.847013,-1.0,62.210273,34.020508,-1.0,6
4,6.691485,-1.000000,3.337971,-1.000000,-1.000000,99.838438,-1.000000,-1.000000,0,1.957666,34.633063,-1.0,34.612121,-1.0,112.411298,20.074456,-1.0,5


In [6]:
len(train_df)

50400

In [7]:
test_df = pd.read_csv('../../final/data/test_set_constant.csv')
test_df.head()

,hemoglobin,ferritin,ret_count,segmented_neutrophils,tibc,mcv,serum_iron,rbc,gender,creatinine,cholestrol,copper,ethanol,folate,glucose,hematocrit,tsat,label
0,7.116363,-1.000000,3.781573,2.738413,-1.000000,95.904198,68.457895,2.226085,0,1.892912,39.808550,110.329197,64.404350,21.654404,73.787009,21.349089,-1.000000,5
1,8.125320,92.230003,4.231419,1.188039,143.365567,104.057204,204.747831,2.342554,0,0.652614,13.478089,-1.000000,32.705481,-1.000000,43.520272,24.375961,142.815207,1
2,11.309450,38.324563,-1.000000,-1.000000,455.077909,76.402602,-1.000000,4.440732,0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,33.928350,-1.000000,4
3,13.763858,253.513394,2.262606,0.551444,453.772884,82.781943,90.101466,4.987993,0,0.853521,104.005514,34.639227,0.963866,22.083012,88.891838,41.291574,19.856071,0
4,11.464002,-1.000000,-1.000000,-1.000000,320.964653,104.287127,-1.000000,3.297819,0,1.163516,121.616315,105.895897,-1.000000,9.337462,-1.000000,34.392007,-1.000000,7


In [8]:
X_train = train_df.iloc[:, 0:-1]
y_train = train_df.iloc[:, -1]

X_test = test_df.iloc[:, 0:-1]
y_test = test_df.iloc[:, -1]

X_train, y_train = np.array(X_train), np.array(y_train)
X_test, y_test = np.array(X_test), np.array(y_test)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((50400, 17), (14000, 17), (50400,), (14000,))

#### Training 

In [9]:
# %%time
# timesteps = int(1e6)
# dqn_model = utils.stable_dqn(X_train, y_train, timesteps, True, f'../../models/many_features/stable_dqn_{timesteps}')
# test_df = utils.evaluate_dqn(dqn_model, X_test, y_test)
# test_df.head()

In [10]:
#missingness
# if level==0.1:
#     steps = int(12e6)
# elif level==0.2:
#     steps = int(10e6)
# elif level==0.3:
#     steps = int(12e6)
# else:
#     print('UNKNOWN LEVEL')
# level, steps

In [11]:
#noisiness
# if level==0.1:
#     steps = int(15e6)
# elif level==0.2:
#     steps = int(15e6)
# elif level==0.3:
#     steps = int(10e6)
# else:
#     print('UNKNOWN LEVEL')
# level, steps
steps = int(15e6)
level, steps

(0.3, 15000000)

In [12]:
# for steps in [int(50e6)]:
#     dqn_model = utils.stable_dqn(X_train, y_train, steps, True, 
#                                  f'../../../anemia_ml4hc/models/complete_models/dueling_ddqn_{steps}',
#                                  checkpoint_folder = '../../../anemia_ml4hc/models/sb/dueling_double_dqn',
#                                  checkpoint_prefix='dddqn_basic')
    
# for steps in [int(15e6)]:
# # for steps in [int(2e3)]:
#     dqn_model = utils.stable_vanilla_dqn(X_train, y_train, steps, True, 
#                                  f'../../../anemia_ml4hc/models/sb/seed_{SEED}/dqn_pr_{steps}')#,
# dqn_model = utils.stable_dueling_ddqn(X_train, y_train, steps, True,
# f'../../../anemia_ml4hc/models/sb/new_missingness_noisiness/seed_{SEED}/dueling_ddqn_pr_noisiness_{level}_{steps}')


dqn_model = utils.stable_dueling_dqn(X_train, y_train, steps, True, 
f'../../../anemia_ml4hc/models/sb/new_missingness_noisiness/seed_{SEED}/dueling_dqn_pr_missingness_{level}_{steps}')

/home/lmuyama/anaconda3/envs/stable_baselines_tf2_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning:

Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.

/home/lmuyama/anaconda3/envs/stable_baselines_tf2_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning:

Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.

/home/lmuyama/anaconda3/envs/stable_baselines_tf2_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning:

Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.

/home/lmuyama/anaconda3/envs/stable_baselines_tf2_env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:529: FutureWa

using just stable baselines (not 3) - dueling dqn
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use tf.cast instead.


Invalid MIT-MAGIC-COOKIE-1 key/home/lmuyama/anaconda3/envs/stable_baselines_tf2_env/lib/python3.7/site-packages/stable_baselines/__init__.py:33: UserWarning:

stable-baselines is in maintenance mode, please use [Stable-Baselines3 (SB3)](https://github.com/DLR-RM/stable-baselines3) for an up-to-date version. You can find a [migration guide](https://stable-baselines3.readthedocs.io/en/master/guide/migration.html) in SB3 documentation.



--------------------------------------
| % time spent exploring  | 98       |
| episodes                | 10000    |
| mean 100 episode reward | -0.8     |
| steps                   | 26498    |
| success rate            | 0.17     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 96       |
| episodes                | 20000    |
| mean 100 episode reward | -0.7     |
| steps                   | 53391    |
| success rate            | 0.14     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 94       |
| episodes                | 30000    |
| mean 100 episode reward | -0.7     |
| steps                   | 80755    |
| success rate            | 0.13     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 93       |
| episodes                | 40000    |
| mean 100 episode reward | -0.7     |
| steps                  

--------------------------------------
| % time spent exploring  | 40       |
| episodes                | 320000   |
| mean 100 episode reward | -0.9     |
| steps                   | 937167   |
| success rate            | 0.14     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 38       |
| episodes                | 330000   |
| mean 100 episode reward | -0.9     |
| steps                   | 966827   |
| success rate            | 0.11     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 36       |
| episodes                | 340000   |
| mean 100 episode reward | -0.9     |
| steps                   | 996256   |
| success rate            | 0.09     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 35       |
| episodes                | 350000   |
| mean 100 episode reward | -0.9     |
| steps                  

...

--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 2800000  |
| mean 100 episode reward | -0.2     |
| steps                   | 9545943  |
| success rate            | 0.5      |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 2810000  |
| mean 100 episode reward | -0       |
| steps                   | 9581674  |
| success rate            | 0.55     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 2820000  |
| mean 100 episode reward | -0.1     |
| steps                   | 9620458  |
| success rate            | 0.5      |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 2830000  |
| mean 100 episode reward | -0.1     |
| steps                  

--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 3110000  |
| mean 100 episode reward | -0.2     |
| steps                   | 10672089 |
| success rate            | 0.46     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 3120000  |
| mean 100 episode reward | -0.2     |
| steps                   | 10709191 |
| success rate            | 0.45     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 3130000  |
| mean 100 episode reward | -0.2     |
| steps                   | 10745679 |
| success rate            | 0.46     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 3140000  |
| mean 100 episode reward | -0.2     |
| steps                  

--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 3420000  |
| mean 100 episode reward | -0.3     |
| steps                   | 11806676 |
| success rate            | 0.45     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 3430000  |
| mean 100 episode reward | -0.2     |
| steps                   | 11842596 |
| success rate            | 0.45     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 3440000  |
| mean 100 episode reward | -0.2     |
| steps                   | 11879221 |
| success rate            | 0.48     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 3450000  |
| mean 100 episode reward | -0.3     |
| steps                  

--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 3730000  |
| mean 100 episode reward | -0.1     |
| steps                   | 12904115 |
| success rate            | 0.52     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 3740000  |
| mean 100 episode reward | -0.3     |
| steps                   | 12938992 |
| success rate            | 0.4      |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 3750000  |
| mean 100 episode reward | -0.1     |
| steps                   | 12974158 |
| success rate            | 0.53     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 3760000  |
| mean 100 episode reward | -0.1     |
| steps                  

--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 4040000  |
| mean 100 episode reward | -0.1     |
| steps                   | 13985312 |
| success rate            | 0.53     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 4050000  |
| mean 100 episode reward | -0.1     |
| steps                   | 14019968 |
| success rate            | 0.57     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 4060000  |
| mean 100 episode reward | -0.1     |
| steps                   | 14054863 |
| success rate            | 0.51     |
--------------------------------------
--------------------------------------
| % time spent exploring  | 5        |
| episodes                | 4070000  |
| mean 100 episode reward | -0.2     |
| steps                  

#### Testing

#### Saving files

In [13]:
# test_df.to_csv(f'../../test_dfs/many_features/test_df_2800000.csv', index=False)
# success_df.to_csv(f'../../test_dfs/many_features/success_df_2800000.csv', index=False)